In [10]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.llms.ollama import Ollama

In [4]:

llm = Ollama(model="llama3", request_timeout=120.0)

In [5]:
resp = llm.complete("Who is Paul Graham?")

In [6]:
resp

CompletionResponse(text="Paul Graham (1924-2011) was a British Anglican priest and missionary who served in various parts of the world, including Africa, Asia, and Europe. He is best known for his work among the Maasai people in Tanzania.\n\nGraham was ordained as an Anglican priest in 1950 and spent much of his career serving with the Church Mission Society (CMS) in Africa. In the early 1960s, he moved to Tanzania (then Tanganyika) to work among the Maasai people, a traditionally pastoralist ethnic group.\n\nGraham's work among the Maasai was marked by his deep respect for their culture and way of life, as well as his commitment to building relationships with local leaders and communities. He learned the Maasai language and became proficient in traditional Maasai customs and practices. Graham also worked to establish schools, healthcare facilities, and other community development projects in the area.\n\nGraham's legacy extends beyond his individual work to the broader impact of his m

In [7]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a - b

In [8]:
multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)

In [11]:
agent = ReActAgent.from_tools([multiply_tool, add_tool, subtract_tool], llm=llm, verbose=True)

In [12]:
response = agent.chat("What is 20+(2*4)? Calculate step by step.")

Thought: I can use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
Thought: Now that I know the result of 2*4 is 8, I'll add it to 20.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
Thought: My final answer is the result of adding 20 and 8, which is indeed 28. I can answer without using any more tools.
Answer: The calculation step by step is 2*4 = 8, then 20 + 8 = 28. The answer is 28.


In [13]:
print(response)

The calculation step by step is 2*4 = 8, then 20 + 8 = 28. The answer is 28.


In [14]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do t